<p> <b>Step-1</b> : Upload the file <b>model-training-code.py</b> to the current working directory of the workbench
    
<p> <b>Step-2</b> : Run the custom training job using <b>aiplatform.CustomTrainingJob()</b> method and use <b>us-docker.pkg.dev/vertex-ai/training/xgboost-cpu.1-1:latest</b> for <b>container_uri</b> parameter as your prebuilt image , because this model uses XGBoost framework

In [8]:
from google.cloud import aiplatform

project_id = "udemy-mlops"
region = "us-central1"
staging_bucket="gs://sid-kubeflow-v1"

aiplatform.init(project=project_id, location=region, staging_bucket=staging_bucket)

job = aiplatform.CustomTrainingJob(
        display_name="coupoon-recommendation-training",
        script_path="model-training-code.py",
        container_uri = "us-docker.pkg.dev/vertex-ai/training/xgboost-cpu.1-1:latest",
        requirements=["gcsfs","category_encoders==2.6.1","imbalanced-learn==0.11.0", "scikit-learn>=0.24.0"]
    )

job.run(
    replica_count=1,
    machine_type="n1-standard-4",
    sync=True
)
job.wait()

Training script copied to:
gs://sid-kubeflow-v1/aiplatform-2023-07-24-13:43:41.421-aiplatform_custom_trainer_script-0.1.tar.gz.
Training Output directory:
gs://sid-kubeflow-v1/aiplatform-custom-training-2023-07-24-13:43:41.569 
View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/3339486262623993856?project=1090925531874
CustomTrainingJob projects/1090925531874/locations/us-central1/trainingPipelines/3339486262623993856 current state:
PipelineState.PIPELINE_STATE_RUNNING
View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/6792849975328899072?project=1090925531874
CustomTrainingJob projects/1090925531874/locations/us-central1/trainingPipelines/3339486262623993856 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob projects/1090925531874/locations/us-central1/trainingPipelines/3339486262623993856 current state:
PipelineState.PIPELINE_STATE_RUNNING
CustomTrainingJob projects/1090925531

<p> <b>Step-3</b> : Upload the Model to model registry by using the <b>aiplatform.Model.upload()</b> method.
    <h4><font color="red">Hint : </font></h4>
       Use <b>us-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.1-6:latest</b> as  <b>serving_container_image_uri</b> 

In [9]:
display_name = "coupon-recommendation-sdk"

artifact_uri = "gs://sid-kubeflow-v1/coupon-recommendation/artifacts/"

serving_container_image_uri = "us-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.1-6:latest"

model = aiplatform.Model.upload(
        display_name=display_name,
        artifact_uri=artifact_uri,
        serving_container_image_uri=serving_container_image_uri,
        sync=False
)

Creating Model
Create Model backing LRO: projects/1090925531874/locations/us-central1/models/8700033089335721984/operations/3287398136208687104
Model created. Resource name: projects/1090925531874/locations/us-central1/models/8700033089335721984@1
To use this Model in another session:
model = aiplatform.Model('projects/1090925531874/locations/us-central1/models/8700033089335721984@1')


<p> <b>Step-4</b> : Deploy the Model to Vertex Endpoint by using the <b>model.deploy()</b> method.

In [10]:
deployed_model_display_name = "coupon-model-endpoint"
traffic_split = {"0": 100}
machine_type = "n1-standard-4"
min_replica_count = 1
max_replica_count = 1

endpoint = model.deploy(
        deployed_model_display_name=deployed_model_display_name,
        traffic_split=traffic_split,
        machine_type=machine_type,
        min_replica_count=min_replica_count,
        max_replica_count=max_replica_count
    )

Creating Endpoint
Create Endpoint backing LRO: projects/1090925531874/locations/us-central1/endpoints/4287000234745135104/operations/3132023949064404992
Endpoint created. Resource name: projects/1090925531874/locations/us-central1/endpoints/4287000234745135104
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/1090925531874/locations/us-central1/endpoints/4287000234745135104')
Deploying model to Endpoint : projects/1090925531874/locations/us-central1/endpoints/4287000234745135104
Deploy Endpoint model backing LRO: projects/1090925531874/locations/us-central1/endpoints/4287000234745135104/operations/7836455972317954048
Endpoint model deployed. Resource name: projects/1090925531874/locations/us-central1/endpoints/4287000234745135104


<p> <b>Step-5</b> : Run online predictions against the endpoint by using <b>endpoint.predict()</b> method.<br>
<b>Note</b> : Even though this is a classification problem , the output value indicates the probability of the output.<br>
Typically any value below or euqal to <b>0.5</b> is considered as <b>class 0</b> and greater than <b>0.5</b> is <b>class 1</b>

In [17]:
INSTANCE =[0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  2,
  3,
  0,
  0,
  3,
  2,
  1,
  0,
  0]
instances_list = [INSTANCE]
prediction = endpoint.predict(instances_list)
print(prediction)

Prediction(predictions=[0.1320988982915878], deployed_model_id='7263305643367333888', model_version_id='1', model_resource_name='projects/1090925531874/locations/us-central1/models/8700033089335721984', explanations=None)


<p> <b>Step-6</b> : Run batch predictions against the uploaded model using the <b>model.batch_predict() </b> <br> method.
    The input csv file <b>test-batch.csv</b> can be found in the same directory 

In [22]:
# model = aiplatform.Model('projects/1090925531874/locations/us-central1/models/8700033089335721984@1')

gcs_input_uri = 'gs://sid-kubeflow-v1/coupon-recommendation/test-batch.csv'
BUCKET_URI = "gs://sid-kubeflow-v1/coupon-recommendation/batch-prediction-output"

batch_predict_job = model.batch_predict(
    job_display_name="coupon_batch_predict",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="csv",
    predictions_format="jsonl",
    machine_type="n1-standard-4",
    starting_replica_count=1,
    max_replica_count=1,
    sync=False
)

Creating BatchPredictionJob
BatchPredictionJob created. Resource name: projects/1090925531874/locations/us-central1/batchPredictionJobs/5473629536044187648
To use this BatchPredictionJob in another session:
bpj = aiplatform.BatchPredictionJob('projects/1090925531874/locations/us-central1/batchPredictionJobs/5473629536044187648')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/5473629536044187648?project=1090925531874
BatchPredictionJob projects/1090925531874/locations/us-central1/batchPredictionJobs/5473629536044187648 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/1090925531874/locations/us-central1/batchPredictionJobs/5473629536044187648 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/1090925531874/locations/us-central1/batchPredictionJobs/5473629536044187648 current state:
JobState.JOB_STATE_RUNNING
